In [1]:
!conda install -y -c conda-forge faiss-gpu
!apt-get -y update
!apt-get -y install libatlas-base-dev

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.14.0
  latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Retrieving notices: ...working... done
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease                
Hit:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease              
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease       
Reading package lists... Done                     
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libatlas-base-dev is already the newest version (3.10.3-5).
0 upgraded, 0 newly installed, 0 to remove and 83 not upgraded.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.random_projection import GaussianRandomProjection

from tqdm import tqdm

import faiss

In [3]:
df = pd.read_csv("story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Blaz...,./hidden_states/prompt_1.npz,270,271
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Spar...,./hidden_states/prompt_1.npz,349,350
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,./hidden_states/prompt_1.npz,278,278
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,117,118
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,129,130
...,...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,289,290
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,119,119
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,127,128
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,441,441


In [4]:
max_story_len = max(df["len_generated_story"])
max_story_len

522

In [5]:
hidden_states_by_layer = {}
NUM_PROMPTS = 2

# for prompt_id in range(1, 11):
for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'./hidden_states/prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"][0]
#             print(curr_hidden_states.shape)
            for layer in range(1):
                padded_arr = np.zeros((max_story_len, 512))
                padded_arr_len = len(curr_hidden_states[layer][0])
                
                padded_arr[:padded_arr_len] = curr_hidden_states[layer][0]
                
                padded_arr = padded_arr.flatten().astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
#                 print(padded_arr.shape)
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(padded_arr)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [padded_arr]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:25<00:00, 39.45it/s]


In [6]:
np.array(hidden_states_by_layer["layer_0"]).shape

(2000, 267264)

In [7]:
# Use original vectors for clustering - uncomment this cell and comment out next cell
# dim_reduced_vecs = np.array(hidden_states_by_layer["layer_0"])

In [8]:
# Layer 0 clustering

random_projector = GaussianRandomProjection(random_state = 42)
dim_reduced_vecs = random_projector.fit_transform(np.array(hidden_states_by_layer["layer_0"])).astype('float32')

In [9]:
dim_reduced_vecs.shape

(2000, 6515)

In [10]:
# K-means Clustering

ncentroids = NUM_PROMPTS
niter = 50
verbose = True
dim = dim_reduced_vecs.shape[1]
kmeans = faiss.Kmeans(dim, ncentroids, niter=niter, verbose=verbose, gpu=True, nredo = 20, spherical = True)
kmeans.train(dim_reduced_vecs)

Sampling a subset of 512 / 2000 for training
Clustering 512 points in 6515D to 2 clusters, redo 20 times, 50 iterations
  Preprocessing in 0.02 s
Outer iteration 0 / 20
  Iteration 49 (0.08 s, search 0.04 s): objective=2476.42 imbalance=1.000 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 20
  Iteration 49 (0.15 s, search 0.07 s): objective=2493.32 imbalance=1.010 nsplit=0       
Objective improved: keep new clusters
Outer iteration 2 / 20


  Iteration 49 (0.23 s, search 0.11 s): objective=2508.96 imbalance=1.009 nsplit=0       
Objective improved: keep new clusters
Outer iteration 3 / 20
  Iteration 49 (0.31 s, search 0.15 s): objective=2509 imbalance=1.008 nsplit=0       
Objective improved: keep new clusters
Outer iteration 4 / 20
  Iteration 49 (0.38 s, search 0.19 s): objective=2472.39 imbalance=1.001 nsplit=0       
Outer iteration 5 / 20


  Iteration 49 (0.46 s, search 0.22 s): objective=2459.67 imbalance=1.010 nsplit=0       
Outer iteration 6 / 20
  Iteration 49 (0.53 s, search 0.26 s): objective=2476.4 imbalance=1.000 nsplit=0       
Outer iteration 7 / 20


  Iteration 49 (0.61 s, search 0.30 s): objective=2492.42 imbalance=1.012 nsplit=0       
Outer iteration 8 / 20
  Iteration 49 (0.69 s, search 0.33 s): objective=2500.1 imbalance=1.000 nsplit=0       
Outer iteration 9 / 20
  Iteration 49 (0.76 s, search 0.37 s): objective=2509.08 imbalance=1.009 nsplit=0       
Objective improved: keep new clusters
Outer iteration 10 / 20


  Iteration 49 (0.84 s, search 0.41 s): objective=2496.8 imbalance=1.006 nsplit=0       
Outer iteration 11 / 20
  Iteration 49 (0.91 s, search 0.44 s): objective=2477.04 imbalance=1.001 nsplit=0       
Outer iteration 12 / 20
  Iteration 49 (0.99 s, search 0.48 s): objective=2509.04 imbalance=1.008 nsplit=0       
Outer iteration 13 / 20


  Iteration 49 (1.07 s, search 0.52 s): objective=2508.34 imbalance=1.008 nsplit=0       
Outer iteration 14 / 20
  Iteration 49 (1.14 s, search 0.56 s): objective=2470.1 imbalance=1.079 nsplit=0       
Outer iteration 15 / 20
  Iteration 49 (1.22 s, search 0.59 s): objective=2490.19 imbalance=1.034 nsplit=0       
Outer iteration 16 / 20


  Iteration 49 (1.30 s, search 0.63 s): objective=2502.35 imbalance=1.000 nsplit=0       
Outer iteration 17 / 20
  Iteration 49 (1.37 s, search 0.67 s): objective=2463.89 imbalance=1.027 nsplit=0       
Outer iteration 18 / 20


  Iteration 49 (1.45 s, search 0.70 s): objective=2460.56 imbalance=1.159 nsplit=0       
Outer iteration 19 / 20


2509.0791015625

In [11]:
kmeans.centroids #cluster centers

array([[-0.00052912,  0.01648908,  0.00235646, ..., -0.00796456,
        -0.00325853, -0.01015502],
       [ 0.02238631,  0.0041292 ,  0.01192847, ..., -0.00827868,
        -0.00809136, -0.01346222]], dtype=float32)

In [12]:
kmeans.obj #inertia at each iteration

array([ 820.02288818, 2419.53466797, 2451.6640625 , 2466.8125    ,
       2472.80883789, 2475.62451172, 2476.42138672, 2476.42138672,
       2476.42138672, 2476.42138672, 2476.42138672, 2476.42138672,
       2476.42138672, 2476.42138672, 2476.42138672, 2476.42138672,
       2476.42138672, 2476.42138672, 2476.42138672, 2476.42138672,
       2476.42138672, 2476.42138672, 2476.42138672, 2476.42138672,
       2476.42138672, 2476.42138672, 2476.42138672, 2476.42138672,
       2476.42138672, 2476.42138672, 2476.42138672, 2476.42138672,
       2476.42138672, 2476.42138672, 2476.42138672, 2476.42138672,
       2476.42138672, 2476.42138672, 2476.42138672, 2476.42138672,
       2476.42138672, 2476.42138672, 2476.42138672, 2476.42138672,
       2476.42138672, 2476.42138672, 2476.42138672, 2476.42138672,
       2476.42138672, 2476.42138672,  748.65246582, 2413.9230957 ,
       2431.6237793 , 2438.2878418 , 2442.21801758, 2446.89453125,
       2450.75170898, 2456.55859375, 2462.72143555, 2474.80859

In [13]:
pd.Series(kmeans.index.search(dim_reduced_vecs.astype(np.float32), 1)[1].flatten()).value_counts()

0    1106
1     894
dtype: int64

In [14]:
classifications = []
for dim_red_vec in dim_reduced_vecs:
    dist_from_first_centroid = np.linalg.norm((dim_red_vec - kmeans.centroids[0]))
    dist_from_second_centroid = np.linalg.norm((dim_red_vec - kmeans.centroids[1]))
    classification = 0 if dist_from_first_centroid < dist_from_second_centroid else 1
    
    classifications.append(classification)
    
pd.Series(classifications).value_counts()

0    1106
1     894
dtype: int64